In [ ]:
from pyincore import IncoreClient, MappingSet, RestorationService, FragilityService
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.epfrestoration import EpfRestoration

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

In [ ]:
# Memphis EPF Damage with Earthquake
hazard_type_eq = "earthquake"
hazard_id_eq = "5b902cb273c3371e1236b36b"
epf_dataset_id = "6267094704ce841cbeb35a97"
mapping_id = "5b47be62337d4a37b6197a3a"  # PGA and PGD

use_liquefaction = True
use_hazard_uncertainty = False
liquefaction_geology_dataset_id = "5a284f53c7d30d13bc08249c"

# Create and setup EPF Damage
epf_dmg_eq_memphis = EpfDamage(client)
epf_dmg_eq_memphis.load_remote_input_dataset("epfs", epf_dataset_id)

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
epf_dmg_eq_memphis.set_input_dataset('dfr3_mapping_set', mapping_set)

epf_dmg_eq_memphis.set_parameter("result_name", "memphis_eq_epf_dmg_result")
epf_dmg_eq_memphis.set_parameter("hazard_type", hazard_type_eq)
epf_dmg_eq_memphis.set_parameter("hazard_id", hazard_id_eq)
epf_dmg_eq_memphis.set_parameter("use_liquefaction", use_liquefaction)
epf_dmg_eq_memphis.set_parameter("use_hazard_uncertainty", use_hazard_uncertainty)
epf_dmg_eq_memphis.set_parameter("liquefaction_geology_dataset_id", liquefaction_geology_dataset_id)
epf_dmg_eq_memphis.set_parameter("num_cpu", 1)

# Run Analysis
epf_dmg_eq_memphis.run_analysis()

In [ ]:
# Create and setup EPF Restoration
epf_rest = EpfRestoration(client)

restoration_table_id = "62d060032812223656f8dbd3"
restoration_mapping_id = "61f302e6e3a03e465500b3eb"

# Load restoration mapping
restorationsvc = RestorationService(client)
mapping_set = MappingSet(restorationsvc.get_mapping(restoration_mapping_id))  # new format of mapping

epf_rest.load_remote_input_dataset('epfs', epf_dataset_id)
epf_rest.set_input_dataset('dfr3_mapping_set', mapping_set)
epf_rest.set_input_dataset('damage', epf_dmg_eq_memphis.get_output_dataset("result"))
epf_rest.load_remote_input_dataset("restoration_table", restoration_table_id)
epf_rest.set_parameter("result_name", "memphis-epf")
epf_rest.set_parameter("restoration_key", "Restoration ID Code")
epf_rest.set_parameter("end_time", 100.0)
epf_rest.set_parameter("time_interval", 1.0)
epf_rest.set_parameter("pf_interval", 0.01)

In [ ]:
# Run Analysis
epf_rest.run_analysis()

In [ ]:
# Discretized Restoration functionality
func_results = epf_rest.get_output_dataset("func_results").get_dataframe_from_csv()
func_results.head()

In [ ]:
# Functionality change over time
pf_results = epf_rest.get_output_dataset("pf_results").get_dataframe_from_csv()
pf_results.head()

In [ ]:
# Repair time at certain functionality recovery and damage state
time_results = epf_rest.get_output_dataset("time_results").get_dataframe_from_csv()
time_results.head()

In [ ]:
from pyincore.analyses.epfrestoration import EpfRestorationUtil

In [ ]:
epf_rest_util = EpfRestorationUtil(epf_rest)

In [ ]:
# Functionality for particular component and damage state
functionality = epf_rest_util.get_percentage_func(guid="75941d02-93bf-4ef9-87d3-d882384f6c10",
                                                      damage_state="DS_0", time=2.0)
functionality

In [ ]:
# Repair time for particular component and damage state
time = epf_rest_util.get_restoration_time(guid="75941d02-93bf-4ef9-87d3-d882384f6c10", damage_state="DS_1", pf=0.81)
time